In [ ]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import requests

In [1]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = bs(source.text, 'lxml')

NameError: name 'requests' is not defined

data_lst = []
columns = []
table = soup.find(class_ = 'wikitable')
for idx, tr_tag in enumerate(table.find_all('tr')):
    part = []
    for td in tr_tag.find_all(['td', 'th']):
        part.append(td.text.strip())
    if not columns: #find header of the table
        columns = part
    else:
        data_lst.append(part)

In [ ]:
ca_df_bs4 = pd.DataFrame(data = data_lst, columns = columns)
ca_df_bs4.head(12)

In [ ]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
table = pd.read_html(source.text)

In [ ]:
ca_df = pd.DataFrame(table[0])
ca_df.head(12)

In [ ]:
ren_set = {'Postal Code': 'PostalCode', 'Borough':'Borough', 'Neighborhood': 'Neighborhood'}
ca_df_temp = ca_df.rename(columns=ren_set)
ca_df_temp.head(12)

In [ ]:
ca_df_temp_1 = ca_df_temp[ca_df_temp['Borough'] != 'Not assigned']
ca_df_temp_1.head(12)

In [ ]:
if True not in ca_df.duplicated(['Postal Code']).tolist():
    print('No duplicates in the Postal Code column.')
else: 
    print('You should combine rows with duplicates in the Postal Code column')

In [ ]:
if ca_df_temp[(ca_df_temp['Neighborhood'].isna())&(ca_df_temp['Borough'] != 'Not assigned')].shape[0] == 0:
    print('Table is empty, no conditions for our case.')

In [ ]:
ca_df_res = ca_df_temp_1.reset_index(drop=True)
ca_df_res

In [ ]:
print('The number of rows in the final dataframe:', ca_df_res.shape[0])

In [ ]:
geo_df = pd.read_csv('Geospatial_Coordinates.csv')
geo_df.head(12)

In [ ]:
ren_set = {'Postal Code': 'PostalCode'}
geo_df_temp = geo_df.rename(columns=ren_set)

In [ ]:
df_merge = pd.merge(ca_df_res, geo_df_temp)
df_merge.head(12)

In [ ]:
#import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
# uncomment this line if you haven't completed the Foursquare API lab
!conda install -c conda-forge geopy --yes

In [ ]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
# uncomment this line if you haven't completed the Foursquare API lab
!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

In [ ]:
to_df = df_merge[df_merge['Borough'].str.contains('Toronto')].reset_index(drop=True)
#to_df.reset_index(drop=True)
to_df.head(12)

In [ ]:

# create map of Toronto using first entries latitude and longitude values
map_to = folium.Map(location=[to_df["Latitude"][0], to_df["Longitude"][0]], zoom_start=10)

In [ ]:
map_to = folium.Map(location=[to_df["Latitude"][0], to_df["Longitude"][0]], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(to_df['Latitude'], to_df['Longitude'], to_df['Borough'], to_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_to)  
    
map_to

In [ ]:
CLIENT_ID = 'MGH5AHE43OSXAFCULZW0ZXBJFXGHOTFW2XILSFJUWWI5XYOM' # your Foursquare ID
CLIENT_SECRET = 'RLNNHMSY14DPZEQMAZIP5AKSM4JHGA0VFKDAS0GTAQ5HLP5S' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


In [ ]:
neighborhood_latitude = to_df['Latitude'][0] # neighborhood latitude value
neighborhood_longitude = to_df['Longitude'][0] # neighborhood longitude value

neighborhood_name = to_df['Neighborhood'][0] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

In [ ]:
# Setup API URL to explore venues near by
LIMIT = 100
radius = 500
url = ('https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'
       .format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT))
neighborhood_json = requests.get(url).json()

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues = neighborhood_json['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(12)

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
to_venues_df = getNearbyVenues(names=to_df['Neighborhood'],
                                   latitudes=to_df['Latitude'],
                                   longitudes=to_df['Longitude']
                                  )

In [ ]:
to_venues_df.head(12)

In [ ]:
to_venues_df.groupby('Neighborhood').count()

In [ ]:
print('There are {} uniques categories.'.format(len(to_venues_df['Venue Category'].unique())))

In [ ]:
# one hot encoding
to_onehot = pd.get_dummies(to_venues_df[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
to_onehot['Neighborhood'] = to_venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [to_onehot.columns[-1]] + list(to_onehot.columns[:-1])
toronto_onehot = to_onehot[fixed_columns]

to_onehot.head(12)

In [ ]:
to_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
to_grouped

In [ ]:
num_top_venues = 5

for hood in to_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = to_grouped[to_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = to_grouped['Neighborhood']

for ind in np.arange(to_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(to_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(12)

In [ ]:
# set number of clusters
kclusters = int(len(to_df["Neighborhood"].unique()) / 4)
to_grouped_clustering = to_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=1).fit(to_grouped_clustering)

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

to_merged = to_df.copy()
to_merged.rename(columns={'Neighbourhood':'Neighborhood'}, inplace=True)
to_merged = to_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

to_merged.head() # check the last columns!

In [ ]:
# create map
map_clusters = folium.Map(location=[to_df["Latitude"][0], to_df["Longitude"][0]], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(to_merged['Latitude'], to_merged['Longitude'], to_merged['Neighborhood'], to_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters